In [ ]:
df = pd.read_csv(".....csv")

In [ ]:
missing_df =df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['Column Name', 'Missing Values Count']
missing_df['Filling Factor (%)']=(df.shape[0]-missing_df['Missing Values Count'])/df.shape[0]*100
missing_df.sort_values('Filling Factor (%)').reset_index(drop = True)
missing_df.to_csv("missing_df")

In [ ]:

starttime=time.time()

from sklearn.impute import KNNImputer
imp = KNNImputer(n_neighbors=5)

X=imp.fit_transform(X)


print("----timetaken is ..... ", (time.time()-starttime))

In [ ]:
import seaborn as sns
plt.figure(figsize= (30,20))  
pos = 1 
for feature in df.columns:   
    plt.subplot(4, 3, pos)   
    if feature in df.columns:   
         sns.distplot(df[feature], kde= True )
    pos += 1 

In [ ]:
df.plot(kind= 'box' , subplots=True,layout=(3,3), sharex=False, sharey=False, figsize=(15,10))

In [ ]:
#outliers by gausian approch and replacing them by mean

for col_name in df.columns:
    mean = df[col_name].mean()
    std = df[col_name].std()
    
    low = mean + 3* std
    high = mean - 3* std
    
    print('outliers of {} (Gaussian approach) are : {}'.format(column,df[(df[col_name] < low) | (df[col_name] > high)].shape[0]/np.float(df.shape[0])))
    
    df.loc[(df[col_name] < low) | (df[col_name] > high), col_name] = df[col_name].mean()

In [ ]:
#Outliers by IQR and Replacing them by median
for col_name in df.columns[:-1]:
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3 - q1
    
    low = q1-1.5*iqr
    high = q3+1.5*iqr
    
    df.loc[(df[col_name] < low) | (df[col_name] > high), col_name] = df[col_name].median()



In [ ]:
#Outliers by IQR and Replacing them by whiskers

def outliers_handling(cols):
    cols = pd.DataFrame(cols)
    Q1 = cols.quantile(0.25)
    Q3 = cols.quantile(0.75)
    IQR = Q3 - Q1
    # print(IQR)

    # Replace every outlier on the lower side by the lower whisker
    for i, j in zip(np.where(cols < Q1 - 1.5 * IQR)[0], np.where(cols < Q1 - 1.5 * IQR)[1]): 

        whisker  = Q1 - 1.5 * IQR
        cols.iloc[i,j] = whisker[j]


    #Replace every outlier on the upper side by the upper whisker    
    for i, j in zip(np.where(cols > Q3 + 1.5 * IQR)[0], np.where(cols > Q3 + 1.5 * IQR)[1]):

        whisker  = Q3 + 1.5 * IQR
        cols.iloc[i,j] = whisker[j]
        
    return cols

for i in df.columns:
    df[i]=outliers_handling(df[i])